In [1]:
import numpy as np
import cv2 as cv

In [2]:
img = cv.imread('img.png')
harris_img = img.copy()
shi_tomachi_img = img.copy()
transformed_img = img

In [3]:
brightness = 10
contrast_value = 10
angle_value = 0.0
scale = 1.0

marker_size = 2

height, width = img.shape[:2]
center = (width / 2, height / 2)

point_list = list()
point_list.append((0, 0))
point_list.append((img.shape[1], 0))
point_list.append((img.shape[1], img.shape[0]))
point_list.append((0, img.shape[0]))
point_for_change = 0
#Harris
block_size = 2
sobel_kernel_size = 3
free_harris_param = 0.001
#Shi-Tomachi
corners_count = 80
corners_quality = 0.05
minimal_distance = 5
show_gray_shi = 0

In [4]:
def update():
	global transformed_img
	transformed_img = img.copy()
	change_brightness(brightness)
	change_contrast(contrast_value)
	rotate_matrix = cv.getRotationMatrix2D(center, angle_value, scale)
	transformed_img = cv.warpAffine(transformed_img, rotate_matrix, (width, height))
	matrix = cv.getPerspectiveTransform(
		np.float32([[0, 0], [img.shape[1], 0], [img.shape[1], img.shape[0]], [0, img.shape[0]]]),
		np.float32(point_list))
	transformed_img = cv.warpPerspective(transformed_img, matrix, (img.shape[1], img.shape[0]), cv.INTER_CUBIC,
										 borderMode=cv.BORDER_CONSTANT, borderValue=(0, 0, 0))
	update_Harris()
	update_Shi_tomachi()


def change_brightness(brightness):
	global transformed_img
	if brightness != 0:
		transformed_img = cv.addWeighted(transformed_img, (255 - brightness) / 255, transformed_img, 0, brightness)


def change_contrast(contrast):
	global transformed_img
	if contrast != 0:
		alpha = 131 * (contrast + 127) / (127 * (131 - contrast))
		transformed_img = cv.addWeighted(transformed_img, alpha, transformed_img, 0, 127 * (1 - alpha))


def change_brightness_size(val):
	global brightness
	brightness = val
	update()


def change_contrast_size(val):
	global contrast_value
	contrast_value = val
	update()


def change_rotate_angle(val):
	global angle_value
	angle_value = val
	update()


def change_rotate_scale(val):
	global scale
	if val > 0:
		scale = val / 10.0
		update()


def change_point_x_val(val):
	global point_list
	_, old_y = point_list[point_for_change]
	point_list[point_for_change] = (val, old_y)
	update()


def change_point_y_val(val):
	global point_list
	old_x, _ = point_list[point_for_change]
	point_list[point_for_change] = (old_x, val)
	update()

# Harris

In [5]:
def update_Harris():
	gray = cv.cvtColor(transformed_img, cv.COLOR_BGR2GRAY)
	corners = cv.cornerHarris(gray, block_size, sobel_kernel_size, free_harris_param)
	harris_img = transformed_img.copy()
	harris_img[corners > 0.01 * corners.max()] = [0, 255, 0]
	cv.imshow('Harris', harris_img)


def change_block_size(val):
	global block_size
	block_size = val
	update_Harris()


def change_sobel_kernel_size(val):
	global sobel_kernel_size
	if val % 2 == 0:
		val += 1
	sobel_kernel_size = val
	update_Harris()


def change_free_harris_param(val):
	global free_harris_param
	if val > 0:
		free_harris_param = val / 100
		update_Harris()

# Shi-Tomachi

In [6]:
def update_Shi_tomachi():
	gray = cv.cvtColor(transformed_img, cv.COLOR_BGR2GRAY)
	corners = cv.goodFeaturesToTrack(gray, corners_count, corners_quality, minimal_distance,
									 mask=cv.inRange(gray, 1, 255))
	shi_tomachi_img = transformed_img.copy()
	corners_int = np.int0(corners)
	for corner in corners_int:
		x, y = corner.ravel()
		cv.circle(shi_tomachi_img, (x,y), marker_size, 255, -1)
	cv.imshow('Shi-Tomachi', shi_tomachi_img)


def change_corners_count(val):
	global corners_count
	if val > 0:
		corners_count = val
		update_Shi_tomachi()


def change_corners_quality(val):
	global corners_quality
	if val > 0:
		corners_quality = val / 100.0
		update_Shi_tomachi()


def change_minimal_distance(val):
	global minimal_distance
	minimal_distance = val
	update_Shi_tomachi()


def change_res_col_shi(val):
	global show_gray_shi
	show_gray_shi = val
	update_Shi_tomachi()


def change_marker_size(val):
	global marker_size
	marker_size = val
	update_Shi_tomachi()

# Let's go!

In [7]:
cv.namedWindow('Settings')
cv.createTrackbar('Brightness', 'Settings', 0, 255, change_brightness_size)
cv.createTrackbar('Contrast','Settings', 0, 128, change_contrast_size)
cv.createTrackbar('Angle', 'Settings', 0, 360, change_rotate_angle)
cv.createTrackbar('Scale', 'Settings', 10, 100, change_rotate_scale)
cv.createTrackbar('X', 'Settings', 0, img.shape[1] - 1, change_point_x_val)
cv.createTrackbar('Y', 'Settings', 0, img.shape[0] - 1, change_point_y_val)
#Harris corners
cv.imshow('Harris', harris_img)
cv.createTrackbar('block size', 'Harris', 2, 50, change_block_size)
cv.createTrackbar('kernel size', 'Harris', 1, 15, change_sobel_kernel_size)
cv.createTrackbar('k', 'Harris', 1, 100, change_free_harris_param)
#Shi tomachi corners
cv.imshow('Shi-Tomachi', shi_tomachi_img)
cv.createTrackbar('corners count', 'Shi-Tomachi', 80, 300, change_corners_count)
cv.createTrackbar('quality', 'Shi-Tomachi', 5, 100, change_corners_quality)
cv.createTrackbar('min distance', 'Shi-Tomachi', 5, 100, change_minimal_distance)
cv.createTrackbar('Marker size', 'Shi-Tomachi', 2, 30, change_marker_size)

cv.waitKey(0)
cv.destroyAllWindows()